# Lensing simulation tutorial

Here we will simulate *CMB lensing*, the process by which the *paths of CMB photons are deflected by large-scale structure as they traverse the Universe.*


The unlensed CMB temperature $\tilde T(\mathbf n)$ at angular position $\mathbf n$ is a Gaussian field, which we have seen how to simulate.

The action of lensing is that we see the CMB evaluated at a *deflected position* $$T^\mathrm{len}(\mathbf n) = \tilde T(\mathbf n + \mathbf d(\mathbf n)).$$

The deflection is (to good approximation) given as the *gradient of a scalar,* $$\mathbf d(\mathbf n)= \nabla \phi(\mathbf n),$$ where $\phi$ is known as the **CMB lensing potential**.  This two-dimensional field is the line-of-sight projection of the 3d gravitational potential from all the strctures in the Universe between us and the CMB.

$\phi$ is (to good approximation) a Gaussian field, and its power spectrum $C_\ell^{\phi\phi}$ is computed by codes like CAMB.  Let's load up the CMB temperature and potential power spectra, and make a simulation of them both.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from cmb_modules import make_map, Plot_Map
from lens_modules import kmask, kappa_to_phi, get_ells, sky2pix, pix2sky, posmap, gradient
np.random.seed(100)

### Simulating an unlensed map



In [ ]:

patch_deg_width = 40. # patch width in degrees
pix_size = 1.5 # pixel size in arcminutes

# Number of pixels in each direction
N = int(patch_deg_width*60./pix_size)

# We need to load the theory spectra
def get_theory():
    ells,tt,_,_,pp,_ = np.loadtxt("CAMB_fiducial_cosmo_scalCls.dat",unpack=True)
    TCMB2 = 7.4311e12
    ckk = pp/4./TCMB2
    ucltt = tt / ells/(ells+1.)*2.*np.pi
    ells2,lcltt = np.loadtxt("CMB_fiducial_totalCls.dat",unpack=True,usecols=[0,1])
    lcltt = lcltt / ells2/(ells2+1.)*2.*np.pi
    lcltt = lcltt[:len(ells)]
    return ells,ucltt,lcltt,ckk


ells,ucltt,lcltt,clkk = get_theory()


In [ ]:

# We next generate an unlensed CMB map as a Gaussian random field as we learned before
DlTT = ucltt*ells*(ells+1.)/2./np.pi
unlensed = make_map(N,pix_size,ells,DlTT)

## variables to set up the map plots
c_min = -400  # minimum for color bar
c_max = 400   # maximum for color bar
X_width = N*pix_size/60.  # horizontal map width in degrees
Y_width = N*pix_size/60.  # vertical map width in degrees


print('Unlensed CMB')
Plot_Map(Map_to_Plot=unlensed,
        c_min=c_min,
        c_max=c_max,
        X_width=X_width,
        Y_width=Y_width)


Sometimes we work with the **CMB lensing convergence** $\kappa(\mathbf n)$, a projection of the matter density.  The potential and convergence are related by a Poisson equation, $$\kappa(\mathbf n) = -\nabla^2 \phi(\mathbf n)/2,$$ or in Fourier space, $$\kappa(\mathbf l) = l^2 \phi(\mathbf l) / 2$$

Let's make a simulation of $\kappa(\mathbf n)$ -- a projection of the (dark) matter between us and the CMB:

In [ ]:
# We also need a lensing convergence (kappa) map
DlKK = clkk*ells*(ells+1.)/2./np.pi
kappa = make_map(N,pix_size,ells,DlKK)
Plot_Map(Map_to_Plot=kappa,
        c_min=None,
        c_max=None,
        X_width=X_width,
        Y_width=Y_width)

To simulate the **deflection** action of lensing, we will obtain our $\phi$ map from this $\kappa$ map; evaluate $\nabla \phi$; and determine the shifted positions for the CMB, $\mathbf n + \nabla \phi(\mathbf n).$  We will do the actual position interpolation using the `scipy.ndimage.map_coordinates`.

In [ ]:
def lens_map(input_map,kappa,modlmap,ly,lx,N,pix_size):
    # First we convert lensing convergence to lensing potential
    phi = kappa_to_phi(kappa,modlmap,return_fphi=True)

    # Then we take its gradient to get the deflection field
    grad_phi = gradient(phi,ly,lx)

    # Then we calculate the displaced positions by shifting the physical positions by the deflections
    deflected_pos = posmap(N,pix_size) + grad_phi

    # We convert the displaced positions into fractional displaced pixel numbers
    # because scipy doesn't know about physical distances
    deflected_pix = sky2pix(deflected_pos,N,pix_size)

    # We prepare an empty output lensed map array
    output_map = np.empty(input_map.shape, dtype=input_map.dtype)

    # We then tell scipy to calculate the values of the input lensed map
    # at the displaced fractional positions by interpolation, and grid that onto the final lensed map
    from scipy.ndimage import map_coordinates
    map_coordinates(input_map, deflected_pix, output_map, order=5, mode='wrap')

    return output_map


In [ ]:

# We get the Fourier coordinates
ly,lx,modlmap = get_ells(N,pix_size)

In [ ]:

# Now we can lens our input unlensed map
lensed = lens_map(input_map=unlensed,
                kappa=kappa,
                modlmap=modlmap,
                ly=ly,
                lx=lx,
                N=N,
                pix_size=pix_size)

In [ ]:

print('Lensed CMB')
Plot_Map(Map_to_Plot=lensed,
        c_min=None,
        c_max=None,
        X_width=X_width,
        Y_width=Y_width)


In [ ]:
print('Lensed minus unlensed CMB')
Plot_Map(Map_to_Plot=lensed - unlensed,
        c_min=c_min/5,
        c_max=c_max/5,
        X_width=X_width,
        Y_width=Y_width) #increase image contrast by a factor of 5

<font color='red'>EXCERCISE: </font> Compare the power spectrum of the lensed CMB map with the unlensed CMB map.

In [ ]:

def calculate_2d_spectrum(Map1,Map2,delta_ell,ell_max,pix_size,N):
    "calculates the power spectrum of a 2d map by FFTing, squaring, and azimuthally averaging"
    N=int(N)
    # make a 2d ell coordinate system
    ones = np.ones(N)
    inds  = (np.arange(N)+.5 - N/2.) /(N-1.)
    kX = np.outer(ones,inds) / (pix_size/60. * np.pi/180.)
    kY = np.transpose(kX)
    K = np.sqrt(kX**2. + kY**2.)
    ell_scale_factor = 2. * np.pi 
    ell2d = K * ell_scale_factor
    
    # make an array to hold the power spectrum results
    N_bins = int(ell_max/delta_ell)
    ell_array = np.arange(N_bins)
    CL_array = np.zeros(N_bins)
    
    # get the 2d fourier transform of the map
    FMap1 = np.fft.ifft2(np.fft.fftshift(Map1))
    FMap2 = np.fft.ifft2(np.fft.fftshift(Map2))
    PSMap = np.fft.fftshift(np.real(np.conj(FMap1) * FMap2))
    # fill out the spectra
    i = 0
    while (i < N_bins):
        ell_array[i] = (i + 0.5) * delta_ell
        inds_in_bin = ((ell2d >= (i* delta_ell)) * (ell2d < ((i+1)* delta_ell))).nonzero()
        CL_array[i] = np.mean(PSMap[inds_in_bin])
        #print i, ell_array[i], inds_in_bin, CL_array[i]
        i = i + 1
 
    # return the power spectrum and ell bins
    return(ell_array,CL_array*np.sqrt(pix_size /60.* np.pi/180.)*2.)

In [ ]:
# Calculate two auto power spectra using the above function: one for the unlensed map, one for the lensed map
# note that the function above takes two maps, but for an auto power spectrum those two maps will be the same

# choose your bin size (delta_ell) and your ell_max (how small of scales you would like to go to, highest possible is 5000 here)

###### Layout: #####

# delta_ell = # choose delta_ell (I suggest between 10-50, but play around with it!)
# ell_max = # choose ell_max < 5000

# ells,unlensed_cl = 
# __,lensed_cl = 

############## YOUR CODE BELOW ############################################


In [ ]:
# Plot them using matplotlib, maybe in multiple ways, to compare.

############## YOUR CODE HERE ############################################